In [1]:
import json
import random
import uuid
import time
from confluent_kafka import Producer
import fastavro
from fastavro.schema import load_schema
import io

# Avro schema (hardcoded)
avro_schema = {
    "type": "record",
    "name": "CreditTransaction",
    "namespace": "com.fraud.detection",
    "fields": [
        {
            "name": "client_id",
            "type": "string"
        },
        {
            "name": "transaction_id",
            "type": "string"
        },
        {
            "name": "type",
            "type": ["string", "null"],
            "default": None
        },
        {
            "name": "amount",
            "type": ["double", "null"],
            "default": None
        },
        {
            "name": "oldbalanceOrg",
            "type": ["double", "null"],
            "default": None
        },
        {
            "name": "newbalanceOrig",
            "type": ["double", "null"],
            "default": None
        }
    ]
}

# Kafka Producer Configuration
KAFKA_BROKER = "host.docker.internal:9092"  # Replace with your Kafka broker
TOPIC = "raw_credit_data"  # Kafka topic to send data to

# Initialize Kafka producer
conf = {
    'bootstrap.servers': KAFKA_BROKER,
    'acks': 'all',  # Wait for all brokers to acknowledge
}

producer = Producer(conf)

# Generate random test data
def generate_test_data():
    client_id = f"client_{random.randint(1000, 9999)}"
    transaction_id = str(uuid.uuid4())
    transaction_type = random.choice([ "TRANSFER"])
    amount = random.uniform(1.0, 10.0)
    old_balance = random.uniform(0.0, 5000.0)
    new_balance = old_balance - amount if transaction_type == "purchase" else old_balance + amount

    return {
        "client_id": client_id,
        "transaction_id": transaction_id,
        "type": transaction_type,
        "amount": amount,
        "oldbalanceOrg": old_balance,
        "newbalanceOrig": new_balance
    }

# Function to serialize data into Avro format
def serialize_avro(data, schema):
    with io.BytesIO() as buf:
        writer = fastavro.writer(buf, schema, [data])
        return buf.getvalue()

# Produce data to Kafka
def produce_to_kafka():
    for _ in range(20):  # Adjust the number of records to send
        test_data = generate_test_data()
        avro_data = serialize_avro(test_data, avro_schema)
        
        # Produce message to Kafka
        producer.produce(topic=TOPIC, key=test_data["transaction_id"], value=avro_data)
        print(f"Sent data: {test_data}")
        producer.flush()
        time.sleep(1)  # Adjust sleep to control message frequency

# Start the producer
if __name__ == "__main__":
    produce_to_kafka()


Sent data: {'client_id': 'client_8771', 'transaction_id': 'e4c71da8-5d2b-42dc-ab4f-e8d67c13ebc4', 'type': 'TRANSFER', 'amount': 8.278315726789963, 'oldbalanceOrg': 4590.569676887551, 'newbalanceOrig': 4598.847992614341}
Sent data: {'client_id': 'client_6090', 'transaction_id': '1b4b8e61-a25b-4924-9f82-7f19949bcbae', 'type': 'TRANSFER', 'amount': 7.400154884905161, 'oldbalanceOrg': 2716.7819350807017, 'newbalanceOrig': 2724.182089965607}
Sent data: {'client_id': 'client_9325', 'transaction_id': '488dd0ca-cada-4a01-8f55-bc219e41e019', 'type': 'TRANSFER', 'amount': 6.373561990399511, 'oldbalanceOrg': 818.7708068429833, 'newbalanceOrig': 825.1443688333828}
Sent data: {'client_id': 'client_6042', 'transaction_id': 'd1bbb892-2517-42f1-9d0d-e8737cc574e8', 'type': 'TRANSFER', 'amount': 3.7647026013915417, 'oldbalanceOrg': 122.45986347718174, 'newbalanceOrig': 126.22456607857329}
Sent data: {'client_id': 'client_9583', 'transaction_id': '0aeb3fea-1771-4cb3-b95e-28c7557c9da2', 'type': 'TRANSFER'

In [6]:
def bellman_ford(graph, start):
    # graph: liste des arcs sous forme (u, v, poids)
    # Pour plus de commodité, on peut transformer le graphe en liste d'arcs.
    nodes = set()
    for u, v, _ in graph:
        nodes.add(u)
        nodes.add(v)
    distances = {node: float('inf') for node in nodes}
    distances[start] = 0
    predecessors = {node: None for node in nodes}
    n = len(nodes)
    
    # Relaxation des arêtes (n-1) fois
    for _ in range(n - 1):
        for u, v, weight in graph:
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
                predecessors[v] = u
    
    # Vérification des cycles négatifs
    for u, v, weight in graph:
        if distances[u] != float('inf') and distances[u] + weight < distances[v]:
            raise ValueError("Le graphe contient un cycle absorbant (cycle de coût négatif)")
    
    return distances, predecessors

# Exemple d'utilisation :
graph_edges = [
    ('A', 'B', 4),
    ('A', 'C', 2),
    ('B', 'C', 3),
    ('B', 'D', 2),
    ('B', 'E', 3),
    ('C', 'B', 1),
    ('C', 'D', 4),
    ('C', 'E', 5),
    ('E', 'D', -5)
]
distances_bf, preds_bf = bellman_ford(graph_edges, 'A')
print("Distances Bellman-Ford:", distances_bf)
print("Prédecesseurs Bellman-Ford:", preds_bf)

Distances Bellman-Ford: {'C': 2, 'B': 3, 'E': 6, 'D': 1, 'A': 0}
Prédecesseurs Bellman-Ford: {'C': 'A', 'B': 'C', 'E': 'B', 'D': 'E', 'A': None}


In [7]:
def topological_sort(graph):
    # Création de la liste des nœuds
    nodes = []
    for node in graph:
        nodes.append(node)
    # Calcul du degré entrant de chaque nœud
    in_degree = {}
    for node in nodes:
        in_degree[node] = 0
    for node in nodes:
        for neighbor_weight in graph[node]:
            neighbor = neighbor_weight[0]
            # Augmente le degré entrant pour le voisin
            in_degree[neighbor] = in_degree.get(neighbor, 0) + 1

    order = []
    # Recherche de nœuds avec degré entrant 0
    while True:
        found = False
        for node in nodes:
            if in_degree[node] == 0 and node not in order:
                order.append(node)
                # Réduire le degré entrant de ses voisins
                for neighbor_weight in graph[node]:
                    neighbor = neighbor_weight[0]
                    in_degree[neighbor] = in_degree[neighbor] - 1
                found = True
        if not found:
            break
    return order

def ford_dag(graph, start):
    # Calcul de l'ordre topologique
    order = topological_sort(graph)
    distances = {}
    predecessors = {}
    for node in graph:
        distances[node] = float('inf')
        predecessors[node] = None
    distances[start] = 0

    # Relaxation dans l'ordre topologique
    for u in order:
        if distances[u] != float('inf'):
            for neighbor_weight in graph[u]:
                neighbor = neighbor_weight[0]
                weight = neighbor_weight[1]
                if distances[u] + weight < distances[neighbor]:
                    distances[neighbor] = distances[u] + weight
                    predecessors[neighbor] = u
    return distances, predecessors

# Exemple de graphe orienté acyclique (DAG)
graph_dag = {
    'A': [('B', 5), ('C', 3)],
    'B': [('D', 2)],
    'C': [('B', 1), ('D', 7)],
    'D': []
}

# Exécution de l'algorithme de Ford pour DAG depuis le sommet 'A'
distances_dag, preds_dag = ford_dag(graph_dag, 'A')

print("\n=== Algorithme de Ford pour DAG ===")
print("Distances DAG :", distances_dag)
print("Prédecesseurs DAG :", preds_dag)


=== Algorithme de Ford pour DAG ===
Distances DAG : {'A': 0, 'B': 4, 'C': 3, 'D': 6}
Prédecesseurs DAG : {'A': None, 'B': 'C', 'C': 'A', 'D': 'B'}
